In [10]:
import pandas as pd
import math
from itertools import tee

Import txt dataset

In [4]:
#used only one time at the start
# df = pd.read_fwf('E:/Rockyou/rockyou.txt', header=None, names=['password'], encoding='latin-1')


convert to csv

In [5]:
#used only one time at the start
# df.to_csv('my_dataframe.csv', index=False)

In [6]:
print(df.head())

    password
0     123456
1      12345
2  123456789
3   password
4   iloveyou


Import the new CSV file

In [7]:
df = pd.read_csv("rockyou.csv", encoding="latin-1")


Extract charactaristic features

num_of_features

In [ ]:
#num of letters
df["num_letters"] = df["password"].str.count(r"[A-Za-z]")
df["num_upper"] = df["password"].str.count(r"[A-Z]")
df["num_lower"] = df["password"].str.count(r"[a-z]")
#num of digits 
df["num_digits"] = df["password"].str.count(r"[0-9]")  
#num of special characters
df["num_special_char"] = df["password"].str.count(r"[^A-Za-z0-9]")

Boolean_features

In [ ]:
#general
df["has_upper"] = df["num_upper"] > 0
df["has_num"] = df["num_didgits"] > 0
df["has_special"] = df["num_special_char"] > 0
#first letter
df["first_is_upper"] = df["password"].str.match(r"^[A-Z]")
df["first_is_digit"] = df["password"].str.match(r"^[0-9]")
df["first_is_special"] = df["password"].str.match(r"^[^A-Za-z0-9]")
#last letter
df["last_is_upper"] = df["password"].str[-1].str.match(r"[A-Z]")
df["last_is_digit"] = df["password"].str[-1].str.match(r"[0-9]")
df["last_is_special"] = df["password"].str[-1].str.match(r"[^A-Za-z0-9]")

ratio_features

In [ ]:
ratio_features

object
0           <class 'str'>
1           <class 'str'>
2           <class 'str'>
3           <class 'str'>
4           <class 'str'>
                ...      
14344375    <class 'str'>
14344376    <class 'str'>
14344377    <class 'str'>
14344378    <class 'str'>
14344379    <class 'str'>
Name: password, Length: 14344380, dtype: object


entropy_features

functions

In [ ]:
def shannon_entropy(pwd):
    if not pwd:
        return 0
    freq = {c: pwd.count(c)/len(pwd) for c in set(pwd)}
    return -sum(p * math.log2(p) for p in freq.values())

df["shannon_entropy"] = df["password"].apply(shannon_entropy)

def ngrams(seq, n=2):
    a, b = tee(seq)
    for _ in range(1, n):
        b = tee(b)[1]
        next(b, None)
    return zip(*a, *b)

def bigram_entropy(pwd):
    pwd = str(pwd)
    if len(pwd) < 2:
        return 0
    bigrams = list(zip(pwd, pwd[1:]))
    freq = {bg: bigrams.count(bg)/len(bigrams) for bg in set(bigrams)}
    return -sum(p * math.log2(p) for p in freq.values())

sequences = ["abcdefghijklmnopqrstuvwxyz", "0123456789", "qwertyuiop", "asdfghjkl", "zxcvbnm"]
def pattern_entropy(pwd):
    pwd = pwd.lower()
    penalty = 0
    for seq in sequences:
        for i in range(len(seq)-2):
            pattern = seq[i:i+3]
            if pattern in pwd:
                penalty += 1
    return max(0, shannon_entropy(pwd) - penalty*0.2)

keyboard_sequences = ["qwertyuiop", "asdfghjkl", "zxcvbnm"]
def keyboard_entropy(pwd):
    pwd = pwd.lower()
    penalty = 0
    for seq in keyboard_sequences:
        for i in range(len(seq)-2):
            if seq[i:i+3] in pwd:
                penalty += 1
    return max(0, shannon_entropy(pwd) - penalty*0.2)

features

In [ ]:
df["shannon_entropy"] = df["password"].apply(shannon_entropy)
df["length_adjusted_entropy"] = df["shannon_entropy"] * df["password"].str.len()
df["bigram_entropy"] = df["password"].apply(bigram_entropy)
df["pattern_entropy"] = df["password"].apply(pattern_entropy)
df["keyboard_entropy"] = df["password"].apply(keyboard_entropy)
